In [2]:
from requests import get, post, put, delete

### With URL Params

In [5]:
url = 'http://localhost:9200/students/_search?q=Minnesota&size=20&pretty' # &_source=false, &_source=name, &_source=name,totalLectures
resp = get(url)
resp.json()

In [ ]:
# Specifying size, and sort
url = 'http://localhost:9200/students/_search?q=montana&sort=age:desc&size=2&pretty'
resp = get(url)
resp.json()

### With Query in request body

In [6]:
url = 'http://localhost:9200/students/_search?pretty' # &_source=false, &_source=name, &_source=name,totalLectures
# data = {'query': { 'match_all': {} }} # or data = {'query': {}} for match all

# data = {'query': { 'match_all': {} },
#         'size': 3}
# Also, If want to query multiple indices or multiple types just include them using comma e.g student,courses

data = {
    'query': { 'match_all': {} },
    'sort': { 'age': { 'order': 'desc' }},
    'from': '10',
    'size': 3
}

resp = post(url, json=data)
resp.json()

### Term Queries

In [43]:
# Queries related to term

# query = {
#     'query': { 'term': { 'name': 'john' } }
# }

# query = {
#     'query': { 'match': { 'name': 'John is' } }
# }

# Difference between term and match is that match will analyze the query first. On the other hand term will
# search the exact same keyword with analyzing it.

# data will not be return
# query = {
#     '_source': False,
#     'query': { 'term': { 'name': 'Hassan' } }
# }

# # only those columns will be returned which starts with "st" or contains "m"
query = {
    '_source': 'st*',
    # or '_source': ['st*', '*m*'],
    'query': { 'term': { 'name': 'Hassan' } }
}

# # OR
# _source = {
#     'includes': ['st*', '*m*'],
#     'excludes': '*il'
# }

resp = post(url, json=query)
resp.json()

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'hits': {u'hits': [],
  u'max_score': None,
  u'total': {u'relation': u'eq', u'value': 0}},
 u'timed_out': False,
 u'took': 11}

### Full Text Search_Match

match

match_phrase

match_phrase_prefix

In [10]:
query = {
    'query': { 'match': { 'name': 'Hassan' } }
}

query = {
    'query': { 'match': { 'name': {
        'query': 'bray durham',
        'operator': 'or'
    } } }
}

# match_phrase - exactly matches "hassan mehmood" in name
query = {
    'query': { 'match_phrase': { 'name': 'Hassan Mehmood' } }
}

# match_phrase_prefix 
query = {
    'query': { 'match_phrase_prefix': { 'name': 'Hass' } }
}
query = {
    'query': { 'match_phrase_prefix': { 'street': 'Monoro Str' } }
}



### Compound Queries
must, should, must_not, filter

In [7]:

# this query means street must contains following 2 words.
query = {
    'query': {
        'bool': {
            'must': [
                {'match': {'street': 'roder'}},
                {'match': {'street': 'avenue'}}
            ]
        }
    }
}

# Should acts like OR
query = {
    'query': {
        'bool': {
            'should': [
                {'match': {'street': 'roder'}},
                {'match': {'street': 'avenue'}}
            ]
        }
    }
}

# must not
query = {
    'query': {
        'bool': {
            'must_not': [
                {'match': {'state': 'arizona kentucky'}},
                {'match': {'street': 'avenue street place lane'}}
            ]
        }
    }
}

resp = post(url, json=query)
resp.json()

### Term Queries Boosting Terms

In [15]:
query = {
    'query': {
        'bool': {
            'should': [{
                'term': {
                    'state': {
                        'value': 'delaware',
                        'boost': 2.0
                    }
                }
            },
            {
                'term': {
                    'state': {
                        'value': 'texas'
                    }
                }
            }]
        }
    }
}

resp = post(url, json=query)
resp.json()

### Filters

In [17]:
query = {
    'query': {
        'bool': {
            'must': { 'match_all': {} },
            'filter': {
                'range': {
                    'range': {
                        'gte': 25,
                        'lte': 35
                    }
                }
            }
        }
    }
}

resp = post(url, json=query)
resp.json()